# *FINAL ASSIGNMENT: BATTLE OF NEIGHBORHOODS*

# GOAL: FIND BEST LOCATIONS FOR AN ITALIAN RESTAURANT IN LONDON

### I Introduction & Problem Statement

### **Problem Statement**:

The goal is to use Data Analysis from web sources in order to chose the best location in London for an Italian restaurant. We will try to take all possible aspects into consideration in order to provide a list of best locations as a suggestion to the entrepreneur.

![Italian Restaurant](https://ta-images.condecdn.net/image/3wA8ZAg7NAN/crop/1020/f/il-pampero-4.jpg)

### **Introduction** 

London is the capital and largest city of England and the United Kingdom. It is considered to be one of the world's most important global cities and has been termed the world's most powerful, most desirable, most influential, most visited, most expensive, innovative, sustainable, most investment friendly and most popular for work city in the world. London exerts a considerable impact upon the arts, commerce, education, entertainment, fashion, finance, health care, media, professional services, research and development, tourism and transportation. London ranks 26 out of 300 major cities for economic performance and is one of the largest financial centers.

London has a diverse range of people and cultures, and more than 300 languages are spoken in the region. Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the most populous of any city in the European Union and accounting for 13.4% of the UK population.

### **Target audience**

Entrepreneur who wants to find the best location to invest or open an Italian restaurant. This location is supposed not to have many Italian restaurants around to avoid competition and must be a convenient neighborhood.  

### **Collecting Data**

For this project we need the following data:
1. London city data that contains Borough, Neighborhoods along with there latitudes and longitudes
* Data Source: https://cocl.us/london_dataset
* Description: This data set contains the required information. We will use this data set to explore various neighborhoods of London.

2. Italian restaurants in each neighborhood of London.
* Data Source: Foursquare API
* Description: By using this API we will get all the venues in different London neighborhoods. We can filter these venues to get only Italian restaurants.

### **Data approach**

We will collect the London city data from https://cocl.us/london_dataset
* Using Foursquare API we will get all venues for each neighborhood.
* Filter out all venues which are Italian Restaurants.
* Data Visualization and some analysis.
* Analysis using k-means clustering:
1. Find the best value of k
2. Visualize the neighborhood with number of Italian Restaurants.
* Compare the Neighborhoods to find the best location for opening a restaurant
* Deductions from these results and conclusions

### II Data Analysis

In [2]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    lib

We will get the London data such as